<a href="https://colab.research.google.com/github/Dinesh-Adhithya-H/Scalable-Machine-Learning-on-Big-Data-using-Apache-Spark/blob/main/supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
import pyspark
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
sc=SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark=SparkSession.builder.getOrCreate()

     |████████████████████████████████| 212.4MB 72kB/s 
     |████████████████████████████████| 204kB 20.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=da6d523241d55e509583a26787068f990f5d9738c4275bb2e55de77234d79005
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [2]:
!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
df = spark.read.parquet('hmp.parquet')
df.createOrReplaceTempView('df')

--2021-06-05 11:38:20--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet [following]
--2021-06-05 11:38:20--  https://github.com/IBM/skillsnetwork/raw/master/hmp.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet [following]
--2021-06-05 11:38:20--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93299

In [3]:
splits=df.randomSplit([0.8,0.2])

In [6]:
train=splits[0]
test=splits[1]

In [16]:
train.show()

+---+---+---+--------------------+-------------+
|  x|  y|  z|              source|        class|
+---+---+---+--------------------+-------------+
|  0| 12| 39|Accelerometer-201...|Sitdown_chair|
|  0| 15| 39|Accelerometer-201...|  Brush_teeth|
|  0| 16| 31|Accelerometer-201...|    Getup_bed|
|  0| 23| 36|Accelerometer-201...|  Brush_teeth|
|  0| 24| 35|Accelerometer-201...|Sitdown_chair|
|  0| 25| 30|Accelerometer-201...|  Brush_teeth|
|  0| 25| 40|Accelerometer-201...|  Brush_teeth|
|  0| 26| 15|Accelerometer-201...| Climb_stairs|
|  0| 26| 42|Accelerometer-201...|  Brush_teeth|
|  0| 27| 31|Accelerometer-201...|Sitdown_chair|
|  0| 27| 33|Accelerometer-201...|    Getup_bed|
|  0| 27| 37|Accelerometer-201...|  Brush_teeth|
|  0| 27| 39|Accelerometer-201...|  Brush_teeth|
|  0| 27| 41|Accelerometer-201...|  Brush_teeth|
|  0| 28| 28|Accelerometer-201...|  Brush_teeth|
|  0| 28| 48|Accelerometer-201...|  Brush_teeth|
|  0| 29| 25|Accelerometer-201...| Climb_stairs|
|  0| 29| 34|Acceler

In [7]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

In [8]:
indexer=StringIndexer(inputCol="class",outputCol="label")
vectorassembler=VectorAssembler(inputCols=["x","y","z"],outputCol="features")
normalizer=Normalizer(inputCol="features",outputCol="features_norm", p=1.0)

In [11]:
from pyspark.ml.classification import LogisticRegression,LinearSVC
from pyspark.ml import Pipeline

In [43]:
lr=LogisticRegression(featuresCol ="features_norm", labelCol = 'label',maxIter=100,regParam=0.5,elasticNetParam=0.8)
pipeline=Pipeline(stages=[indexer,vectorassembler,normalizer])

In [44]:
model=pipeline.fit(train)
prediction=model.transform(test)

In [45]:
prediction

DataFrame[x: int, y: int, z: int, source: string, class: string, label: double, features: vector, features_norm: vector]

In [46]:
prediction.printSchema()

root
 |-- x: integer (nullable = true)
 |-- y: integer (nullable = true)
 |-- z: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- class: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- features_norm: vector (nullable = true)

